In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
import pandas_profiling as pp

import os


import sys
sys.path.append("../")

from src.data.google_storage_io import read_csv_data

In [7]:
# input parameters cell
input_file = "gs://lvt-schiphol-assignment-snakemake/data/raw/flights.csv"
output_file = "processed_flights.csv"

### Read data

In [8]:
df = read_csv_data(input_file)
print(f"Loaded data from: {input_file}\n"
      f"Shape of data: {df.shape}")

Reading file from Google Storage
Bucket:	lvt-schiphol-assignment-snakemake
File:	data/raw/flights.csv

Loaded data from: gs://lvt-schiphol-assignment-snakemake/data/raw/flights.csv
Shape of data: (523275, 28)


In [9]:
df.head()

,actualOffBlockTime,aircraftRegistration,aircraftType.iatamain,aircraftType.iatasub,airlineCode,baggageClaim,estimatedLandingTime,expectedTimeBoarding,expectedTimeGateClosing,expectedTimeGateOpen,...,prefixICAO,publicEstimatedOffBlockTime,publicFlightState.flightStates,route.destinations,scheduleDate,scheduleTime,serviceType,terminal,transferPositions,transferPositions.transferPositions
0,NaN,NaN,NaN,NaN,148.0,NaN,NaN,NaN,NaN,NaN,...,ZXP,NaN,['SCH'],['AMS'],2018-01-01,03:02:07,P,NaN,NaN,NaN
1,NaN,PHPXY,AW1,NaN,148.0,NaN,NaN,NaN,NaN,NaN,...,ZXP,NaN,['SCH'],['AMS'],2018-01-01,03:16:00,NaN,NaN,NaN,NaN
2,NaN,NaN,AW1,NaN,148.0,NaN,NaN,NaN,NaN,NaN,...,ZXP,NaN,['SCH'],['AMS'],2018-01-01,03:16:29,P,NaN,NaN,NaN
3,2018-01-01T03:22:00.000+01:00,PHPXB,NaN,NaN,148.0,NaN,NaN,NaN,NaN,NaN,...,ZXP,NaN,['DEP'],['AMS'],2018-01-01,03:30:00,NaN,NaN,NaN,NaN
4,2018-01-01T05:58:22.000+01:00,PHHSJ,73H,73H,164.0,NaN,NaN,NaN,NaN,NaN,...,TRA,NaN,['DEP'],['SPC'],2018-01-01,06:00:00,J,1.0,NaN,NaN


## Data description

- <font color='red'>**Flight Direction (flightDirection):**</font>  
Indicates whether the commercial flight is a departure flight (d) or an arrival flight (a).  

- <font color='red'>**Flightname (flightName):**</font>    
The flight name of the commercial flight as indicated on the ticket of the passenger.  

- <font color='red'>**Flightnumber (flightNumber):**</font>    
Nummeric part of the flight name.  

- <font color='red'>**IATA flight number prefix (prefixIATA):**</font>    
The two character IATA prefix of the airline that operates the commercial flight.  

- <font color='red'>**ICAO flight number prefix (prefixICAO):**</font>    
The three character ICAO prefix of the airline that operates the commercial flight.  

- <font color='red'>**Schedule date (scheduleDate):**</font>   
The date on which the scheduled commercial flight will be operated.  

- <font color='red'>**Schedule time (scheduleTime):**</font>    
The time the commercial flight is scheduled to depart or arrive.  

- <font color='red'>**Service type of a flight. (serviceType):**</font>    
Category of the commercial flight. J = Passenger Line, C= Passenger Charter, F = Freight Line and H = Freight Charter.

- <font color='red'>**Main flight (mainFlight):**</font>    
In case of a codeshare (one flight with several flight numbers of different airlines) the main flight shows the flight number of the airline that actually operates the flight.  

- <font color='red'>**Codeshares (codeshares):**</font>    
All the flight numbers that are related to the main flight.  

- <font color='red'>**Route (route):**</font>    
The complete route of the commercial flight. In most cases this is a direct route between two airports. There are actually also flights that have more than one stopover. Example: GCK-AMS-LGW (Jakarta – Amsterdam – London Gatwick)  

- <font color='red'>**Public Flightstate (publicFlightState):**</font>    
The status of a public flight  

- <font color='red'>**Terminal (terminal):**</font>    
The section of the Schiphol terminal from which the commercial flight will be leaving or will arrive.  

- <font color='red'>**Gate (gate):**</font>    
The number of the gate from which the scheduled flight will depart and the passengers will be boarding.  

- <font color='red'>**Baggage belt (baggageClaim):**</font>    
The luggage belt number in the reclaim hall (reclaim 1,2,3 or 4) the arrived passengers can pick up their luggage.  

- <font color='red'>**Check-in desks (checkinAllocations):**</font>    
The check-in desks that are allocated for the flight (or flights in case of a codeshare).  

- <font color='red'>**Check-in class:**</font>    
The flight classes that are allocated for a specific flight.  

- <font color='red'>**Transfer desks (transferPositions):**</font>    
The transfer desks that are allocated to the flight.  

- <font color='red'>**Aircraft Type, IATA main (aircraftType, iatamain):**</font>    
The IATA main category code of the aircraft that operates the main flight.  

- <font color='red'>**Aircraft Type, IATA sub (aircraftType, iatasub):**</font>    
The IATA sub category code of the aircraft that operates the main flight.  

- <font color='red'>**Aircraft registration (aircraftRegistration):**</font>    
The unique alphanumeric string that identifies the aircraft that operates the main scheduled flight.  

- <font color='red'>**Estimated Landing Time, ELDT. (estimatedLandingTime):**</font>    
The expected time the (arriving) flight will be landing at Amsterdam Airport Schiphol.  

- <font color='red'>**Actual Landing Time, ALDT. (actualLandingTime):**</font>    
The time the aircraft landed on the runway.  

- <font color='red'>**Public estimated off block time (publicEstimatedOffBlockTime):**</font>    
The expected departure time of a departing flight that is made publically. Made visible on the flight displays in the terminal, on the website, on the Schiphol app, on the information Terminals in the lounges etc.  

- <font color='red'>**Actual Off Block time, AOBT (actualOffBlockTime):**</font>    
The actual time of departure of a flight from Amsterdam Airport Schiphol.  

- <font color='red'>**Expected time gate open (expectedTimeGateOpen):**</font>    
The time the gated is expected to be opened by the flight handler.  

- <font color='red'>**Expected time boarding (expectedTimeBoarding):**</font>    
The expected time the boarding of passengers for a flight will start.  

- <font color='red'>**Expected time closing (expectedTimeGateClosing):**</font>  
The time it is expected that the gate will be closed by the flight handler.

- <font color='red'>**Expected time on belt (expectedTimeOnBelt):**</font>    
The time it is expected that the first luggage of a flight will be on the reclaim belt.  

- <font color='red'>**Airline (airlineCode):**</font>    
NVLS code of the airline  

- <font color='red'>**Routes Eu (routesEu):**</font>    
S (Schengen), E (Europe) or N (non-Europe)  

- <font color='red'>**Visa (visa):**</font>    
Destination requires visa (true) or not (false)  

- <font color='red'>**Last Updated (lastUpdated):**</font>    
Time of last update to flight in flight information system.  

- <font color='red'>**Flight ID (id):**</font>    
Unique identifier of the flight.  

- <font color='red'>**Schema Version (schemaVersion):**</font>    
Schema version of the API.  

In [10]:
df["id"].nunique()

512204

In [11]:
df["flightName"].nunique()

7666

In [12]:
def missing_values_percentages(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})

    missing_value_df = missing_value_df.sort_values('percent_missing', ascending=False)
    return missing_value_df

In [13]:
df_missing_prc = missing_values_percentages(df.query("serviceType == 'J'"))
df_missing_prc.plot(kind='bar', x="column_name", y="percent_missing")

In [14]:
import datetime

In [24]:


def check_col_singular(x):
    """check if pd.Series contains more than 1 unique value excluding NaN"""
    return x.dropna().nunique() > 1

def drop_singular_columns(df, verbose=False):
    """Drop DataFrame columns with 1 or fewer unique values excluding NaN"""
    col_singular = df.apply(check_col_singular, axis=0)
    if verbose:
        n_singular = sum(not col_singular)
        
    df_output = df[[col for col, is_valid in col_singular.items() if is_valid]]
    return df_output


In [32]:
%%time

def preprocess_flights(df_flights):
    df = df_flights
    df = df.dropna(subset=["scheduleDate", "scheduleTime", "actualOffBlockTime"]).reset_index()
    # remove singular
    df = drop_singular_columns(df)
    
    # format datetime fields
   
    df["actualOffBlockTime"] = pd.to_datetime(df["actualOffBlockTime"], utc=True).dt.tz_convert('Europe/Amsterdam')
    
    series_datetime_str = df['scheduleDate'].astype(str) + " " + df['scheduleTime'].astype(str)
    df["scheduleDateTime"] = pd.to_datetime(series_datetime_str, format="%Y%m%d %H:%M:%S").dt.tz_localize('Europe/Amsterdam')
    
#     calculate delay as difference between scheduled and realized departure
    df["scheduleDelaySeconds"] = pd.to_timedelta(df["actualOffBlockTime"] - df["scheduleDateTime"]).dt.total_seconds()

    return df
    
df_preprocessed = preprocess_flights(df)
df_preprocessed.head()

Wall time: 5.68 s


,index,actualOffBlockTime,aircraftRegistration,aircraftType.iatamain,aircraftType.iatasub,airlineCode,expectedTimeBoarding,expectedTimeGateClosing,expectedTimeGateOpen,flightName,...,publicEstimatedOffBlockTime,publicFlightState.flightStates,route.destinations,scheduleDate,scheduleTime,serviceType,terminal,transferPositions.transferPositions,scheduleDateTime,scheduleDelaySeconds
0,3,2018-01-01 03:22:00+01:00,PHPXB,NaN,NaN,148.0,NaN,NaN,NaN,ZXP022,...,NaN,['DEP'],['AMS'],2018-01-01,03:30:00,NaN,NaN,NaN,2018-01-01 03:30:00+01:00,-480.0
1,4,2018-01-01 05:58:22+01:00,PHHSJ,73H,73H,164.0,NaN,NaN,NaN,HV5641,...,NaN,['DEP'],['SPC'],2018-01-01,06:00:00,J,1.0,NaN,2018-01-01 06:00:00+01:00,-98.0
2,5,2018-01-01 06:00:00+01:00,PHHSG,73H,73H,100.0,NaN,NaN,NaN,KL2533,...,NaN,['DEP'],['LPA'],2018-01-01,06:05:00,J,1.0,NaN,2018-01-01 06:05:00+01:00,-300.0
3,6,2018-01-01 06:00:00+01:00,PHHSG,73H,73H,164.0,NaN,NaN,NaN,HV6455,...,NaN,['DEP'],['LPA'],2018-01-01,06:05:00,J,1.0,NaN,2018-01-01 06:05:00+01:00,-300.0
4,7,2018-01-01 06:26:34+01:00,PHHXB,73H,73H,164.0,NaN,NaN,NaN,HV5801,...,NaN,['DEP'],['TLV'],2018-01-01,06:15:00,J,1.0,NaN,2018-01-01 06:15:00+01:00,694.0


In [ ]:
df_preprocessed["scheduleDelaySeconds"] = pd.to_timedelta(df_preprocessed["actualOffBlockTime"] - df_preprocessed["scheduleDateTime"]).dt.total_seconds()